In [1]:
# Dependencies:

# Data Science
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# Reflect Tables into SQLAlchemy ORM

In [2]:
# Create engine using the `hawaii.sqlite` database file
engine = create_engine("sqlite:///Resources/hawaii.sqlite")


In [3]:
# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(autoload_with=engine)

In [4]:
# Print all of the classes mapped to the Base
print(list(Base.classes))

[<class 'sqlalchemy.ext.automap.station'>, <class 'sqlalchemy.ext.automap.measurement'>]


In [5]:
# Assign the measurement class to a variable called `Measurement` and
# the station class to a variable called `Station`
Measurement = Base.classes.measurement
Station = Base.classes.station

In [6]:
# Create a session
session = Session(engine)

# Exploratory Precipitation Analysis

In [10]:
# Find the most recent date in the data set.
session.query(Measurement.date).order_by(Measurement.date.desc()).limit(1).all()

[('2017-08-23',)]

In [14]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results.
# Starting from the most recent data point in the database.
from datetime import date
from dateutil.relativedelta import relativedelta

# Calculate the date one year from the last date in data set.
latest_date = date(2017, 8, 23)

print(f"Latest Date: {latest_date}")

start_date = latest_date - relativedelta(months=12)

print(f"Start Date: {start_date}")



Latest Date: 2017-08-23
Start Date: 2016-08-23


In [30]:
# Perform a query to retrieve the data and precipitation scores
prec_scores = session.query(Measurement.date, Measurement.prcp, Measurement.station).\
    filter(Measurement.date >= start_date).\
    order_by(Measurement.date.asc()).\
    all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names
df2 = pd.DataFrame(prec_scores, columns=["Date", "Precipitation", "Station"])

# Sort the dataframe by date
df2["Date"] = pd.to_datetime(df2['Date'])
df2 = df2.sort_values(by="Date", ascending=True).reset_index(drop=True)

# Use Pandas Plotting with Matplotlib to plot the data
df2.head(20)

,Date,Precipitation,Station
0,2016-08-23,0.00,USC00519397
1,2016-08-23,0.15,USC00513117
2,2016-08-23,0.05,USC00514830
3,2016-08-23,NaN,USC00517948
4,2016-08-23,0.02,USC00519523
5,2016-08-23,1.79,USC00519281
6,2016-08-23,0.70,USC00516128
7,2016-08-24,1.45,USC00516128
8,2016-08-24,2.15,USC00519281
9,2016-08-24,1.22,USC00519523


In [24]:
# Use Pandas to calculate the summary statistics for the precipitation data
df2[["Precipitation"]].describe()

,Precipitation
count,2021.000000
mean,0.177279
std,0.461190
min,0.000000
25%,0.000000
50%,0.020000
75%,0.130000
max,6.700000


# Exploratory Station Analysis

In [164]:
# Perform a query to retrieve the data and precipitation scores
stations = session.query(Measurement.date, Measurement.station, Measurement.tobs).\
    order_by(Measurement.date.asc()).\
    all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names
df3 = pd.DataFrame(stations, columns=["Date", "Station", "Temperature"])
df3.head()

,Date,Station,Temperature
0,2010-01-01,USC00519397,65.0
1,2010-01-01,USC00513117,67.0
2,2010-01-01,USC00518838,72.0
3,2010-01-01,USC00519523,75.0
4,2010-01-01,USC00519281,70.0


In [80]:
# Design a query to calculate the total number of stations in the dataset
df3['Station'].nunique()

9

In [160]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
station_ct = df3[["Station"]].value_counts(ascending=False)
print(station_ct)


Station    
USC00519281    2772
USC00519397    2724
USC00513117    2709
USC00519523    2669
USC00516128    2612
USC00514830    2202
USC00511918    1979
USC00517948    1372
USC00518838     511
Name: count, dtype: int64


In [177]:
highest_station = ["USC00519281"]
df4 = df3[df3['Station'].isin(highest_station)].reset_index()


print(df4)

      index        Date      Station  Temperature
0         4  2010-01-01  USC00519281         70.0
1        11  2010-01-02  USC00519281         62.0
2        18  2010-01-03  USC00519281         74.0
3        25  2010-01-04  USC00519281         75.0
4        31  2010-01-05  USC00519281         74.0
...     ...         ...          ...          ...
2767  19513  2017-08-14  USC00519281         77.0
2768  19518  2017-08-15  USC00519281         77.0
2769  19522  2017-08-16  USC00519281         76.0
2770  19526  2017-08-17  USC00519281         76.0
2771  19530  2017-08-18  USC00519281         79.0

[2772 rows x 4 columns]


In [178]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

min_temp = (f"Minimum Temperature {df4['Temperature'].min()}")
max_temp = (f"Maximum Temperature {df4['Temperature'].max()}")
avg_temp = (f"Average Temperature {df4['Temperature'].mean()}")

print(f"Station USC00519281 {min_temp}")
print(f"Station USC00519281 {max_temp}")
print(f"Station USC00519281 {avg_temp}")

Station USC00519281 Minimum Temperature 54.0
Station USC00519281 Maximum Temperature 85.0
Station USC00519281 Average Temperature 71.66378066378067


In [185]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

df4_latest_date = df4["Date"].max()
df4_start_date = df4_latest_date - relativedelta(months=12)
print(df4_start_date)

2016-08-18


# Close Session

In [16]:
# Close Session
session.close()
